### Load the API Keys

In [ ]:
from dotenv import load_dotenv

# from langchain_anthropic import ChatAnthropic
# from langchain_aws import ChatBedrock
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

True

### Load Model

In [ ]:
### for this notebook, we use the Google Generative AI model
# Uncomment the following line to use Anthropic's Claude model instead or another model

# llm = ChatAnthropic(model="claude-3.5-sonnet-20240620", anthropic_api_key="...")
# llm = ChatBedrock( model_id=anthropic.claude-3-5-sonnet-20241022-v2:0, region_name="us-west-2")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

### Simple Sequential Chain:

In [3]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("Write a 3 line poem on {topic}")
chain = prompt | llm | StrOutputParser()
result = chain.invoke({"topic": "autumn"})
print(result)

Golden leaves are falling down,
Crisp air whispers through the town,
Nature's beauty wears a crown.


### Sequential Chain

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt1 = PromptTemplate(
    template='Generate a detailed report on {topic}',
    input_variables=['topic']
)
prompt2 = PromptTemplate(
    template='Generate a 2-point summary from the following text \n {text}',
    input_variables=['text']
)
parser = StrOutputParser()
chain = prompt1 | llm | parser | prompt2 | llm | parser
result = chain.invoke({'topic': 'Global Recession'})
print(result)

Here's a 2-point summary of the provided text:

*   **Global recessions are significant declines in worldwide economic activity, characterized by widespread contractions in GDP, trade, investment, and employment, often triggered by financial crises, demand/supply shocks, policy mistakes, or structural imbalances.**
*   **Effective policy responses, including coordinated monetary and fiscal measures, financial sector reforms, and international cooperation, are crucial to mitigating the impacts of global recessions and promoting a sustainable recovery, especially given the current economic climate of high inflation, geopolitical tensions, and supply chain disruptions.**


In [5]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *      

### Parallel Chain

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel

# Initialize model and parser
parser = StrOutputParser()

# Step 1a: Summary prompt
summary_prompt = PromptTemplate.from_template("Summarize this topic:\n{text}")
summary_chain = summary_prompt | llm | parser

# Step 1b: Fun fact prompt
fact_prompt = PromptTemplate.from_template("Give a fun fact about:\n{text}")
fact_chain = fact_prompt | llm | parser

# Run both in parallel
parallel = RunnableParallel({
    "summary": summary_chain,
    "fact": fact_chain
})

# Step 2: Merge both outputs
merge_prompt = PromptTemplate.from_template(
    "Here is a short summary and a fun fact combined:\nSummary: {summary}\nFun Fact: {fact}"
)
merge_chain = merge_prompt | llm | parser

# Final full chain
chain = parallel | merge_chain

# Sample input
text = "Photosynthesis is the process by which green plants use sunlight to make food from carbon dioxide and water."

# Run
result = chain.invoke({"text": text})
print(result)

This is a great, memorable way to present the information! The summary is concise and accurate, and the fun fact is engaging and humorous. The "plant farts" analogy is certainly attention-grabbing and makes the concept of oxygen being a byproduct of photosynthesis much more memorable. Well done!


In [8]:
# Visualize the chain
chain.get_graph().print_ascii()

                    +-----------------------------+                    
                    | Parallel<summary,fact>Input |                    
                    +-----------------------------+                    
                       ***                   ***                       
                   ****                         ****                   
                 **                                 **                 
    +----------------+                          +----------------+     
    | PromptTemplate |                          | PromptTemplate |     
    +----------------+                          +----------------+     
             *                                           *             
             *                                           *             
             *                                           *             
+------------------------+                  +------------------------+ 
| ChatGoogleGenerativeAI |                  | ChatGoogleGenerati

### Conditional Chain:

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from pydantic import BaseModel, Field
from typing import Literal

# llm = ChatOpenAI()
text_parser = StrOutputParser()
# 1. Define structured output schema
class Feedback(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(
        description="Classify the feedback sentiment"
    )
structured_parser = PydanticOutputParser(pydantic_object=Feedback)
# 2. Classification prompt (with parser instruction)
classification_prompt = PromptTemplate(
    template=(
        "Analyze the feedback and classify it as 'positive' or 'negative'.\n"
        "Feedback: {feedback}\n\n"
        "{format_instruction}"
    ),
    input_variables=["feedback"],
    partial_variables={"format_instruction": structured_parser.get_format_instructions()}
)
# Chain: classify → structured output
classifier_chain = classification_prompt | llm | structured_parser
# 3. Conditional branches for response generation
prompt1 = PromptTemplate.from_template(
          "Write a warm thank-you message for this feedback:\n{feedback}"
           )
positive_response = prompt1 | llm | text_parser
prompt2 = PromptTemplate.from_template(
          "Write a polite and helpful apology response for this feedback:\n{feedback}"
          )
negative_response = prompt2 | llm | text_parser
# 4. Conditional logic using RunnableBranch
response_branch = RunnableBranch(
    (lambda x: x.sentiment == "positive", positive_response),
    (lambda x: x.sentiment == "negative", negative_response),
    RunnableLambda(lambda _: "Sentiment could not be classified.")
)
# 5. Final chain: classify → choose response
chain = classifier_chain | response_branch
# Example usage
feedback_text = "The product arrived late and the box was damaged."
print(chain.invoke({"feedback": feedback_text}))

Okay, I understand. I'm very sorry to hear that you had a negative experience. I truly value your feedback, as it helps me learn and improve. Could you please provide me with more details about what went wrong? Knowing the specifics will allow me to address the issue and prevent it from happening again in the future.

In the meantime, please accept my sincere apologies. I am committed to doing better.
